# Strategy Optimization Workshop

Optimize strategy parameters using genetic algorithms and validate with walk-forward analysis.

## What This Notebook Does:
1. Load top strategies from generation phase
2. Optimize parameters using genetic algorithms
3. Validate with walk-forward analysis
4. Run Monte Carlo simulations
5. Select robust strategies for deployment

In [ ]:
# Setup
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
from strategy_factory.optimizer import StrategyOptimizer
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 100)

print("✅ Imports successful")

## 1. Load Data

In [ ]:
# Load historical data
df = pd.read_csv('../data/crypto/BTCUSD_5m.csv')
df.columns = df.columns.str.lower()

if 'date' in df.columns:
    df = df.rename(columns={'date': 'timestamp'})

df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.set_index('timestamp')

print(f"📊 Loaded {len(df):,} bars")
df.head()

## 2. Optimize SMA Strategy

In [ ]:
# Initialize optimizer
optimizer = StrategyOptimizer(initial_capital=10000, commission=0.001)

# Optimize SMA parameters
sma_result = optimizer.optimize_sma(
    df=df,
    fast_range=(5, 30),
    slow_range=(40, 200),
    generations=30,
    population=50,
    verbose=True
)

print(f"\n🏆 Best SMA parameters: {sma_result.best_params}")
print(f"   Sharpe ratio: {sma_result.best_fitness:.2f}")

In [ ]:
# Plot convergence
plt.figure(figsize=(10, 6))
plt.plot(sma_result.convergence_history)
plt.title('Optimization Convergence - SMA Strategy')
plt.xlabel('Generation')
plt.ylabel('Best Fitness (Sharpe Ratio)')
plt.grid(True)
plt.show()

## 3. Optimize RSI Strategy

In [ ]:
# Optimize RSI parameters
rsi_result = optimizer.optimize_rsi(
    df=df,
    period_range=(5, 30),
    oversold_range=(15, 40),
    overbought_range=(60, 85),
    generations=30,
    population=50,
    verbose=True
)

print(f"\n🏆 Best RSI parameters: {rsi_result.best_params}")
print(f"   Sharpe ratio: {rsi_result.best_fitness:.2f}")

## 4. Walk-Forward Analysis

In [ ]:
# Run walk-forward analysis on best SMA strategy
wf_results = optimizer.walk_forward_analysis(
    df=df,
    strategy_params=sma_result.best_params,
    train_window=2000,  # ~7 days of 5min data
    test_window=500,    # ~1.7 days
    step_size=200,      # ~14 hours
    verbose=True
)

wf_results

In [ ]:
# Plot walk-forward results
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Returns per fold
axes[0].bar(wf_results['fold'], wf_results['test_return'])
axes[0].axhline(y=0, color='r', linestyle='--')
axes[0].set_title('Walk-Forward Returns by Fold')
axes[0].set_xlabel('Fold')
axes[0].set_ylabel('Return (%)')
axes[0].grid(True, alpha=0.3)

# Sharpe per fold
axes[1].bar(wf_results['fold'], wf_results['test_sharpe'], color='green')
axes[1].axhline(y=0, color='r', linestyle='--')
axes[1].set_title('Walk-Forward Sharpe Ratio by Fold')
axes[1].set_xlabel('Fold')
axes[1].set_ylabel('Sharpe Ratio')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Monte Carlo Simulation

In [ ]:
# Run Monte Carlo simulation
mc_results = optimizer.monte_carlo_simulation(
    df=df,
    strategy_params=sma_result.best_params,
    n_simulations=1000,
    confidence_level=0.95,
    verbose=True
)

# Display results
for key, value in mc_results.items():
    print(f"{key:.<30} {value:.2f}%")

## 6. Save Optimized Strategies

In [ ]:
# Collect optimized strategies
optimized_strategies = [
    {
        'strategy': 'SMA Crossover',
        'params': sma_result.best_params,
        'sharpe': sma_result.best_fitness,
        'walk_forward_avg_return': wf_results['test_return'].mean(),
        'walk_forward_avg_sharpe': wf_results['test_sharpe'].mean(),
        'monte_carlo_mean': mc_results['mean_return'],
        'monte_carlo_prob_positive': mc_results['probability_positive']
    },
    {
        'strategy': 'RSI Mean Reversion',
        'params': rsi_result.best_params,
        'sharpe': rsi_result.best_fitness,
    }
]

df_optimized = pd.DataFrame(optimized_strategies)
df_optimized.to_csv('../results/optimized_strategies.csv', index=False)

print("✅ Optimized strategies saved to results/optimized_strategies.csv")
df_optimized

## Next Steps

1. **Analyze performance** - Use `03_performance_analysis.ipynb` for detailed metrics
2. **Deploy to brokers** - Use `04_multi_broker_deployment.ipynb` to go live